In [1]:
from pathlib import Path
from pprint import pprint
from ost import Sentinel1Batch
from ost.helpers import vector, raster
import json
import gdal
import os

# load username and pw to access sentinel files
username = json.load(open('login_data.json'))['username']
pw = json.load(open('login_data.json'))['pw']

# define the project directory
project_dir = Path.home() / "Documents" / "Python" / "sen1_ost" / "data"

In [2]:
# aoi file name
aoi_file = 'test_shape2.geojson'

# define the start and end date
start = "2020-05-01"
end = "2020-05-31"

# define product
key = "ASCENDING_VVVH"

In [ ]:
# create buffer around aoi
vector.buffer_shape(str(project_dir) + '/aoi/' + aoi_file, str(project_dir) + '/aoi/aoi_tmp' ,buffer=0.01)
aoi = vector.aoi_to_wkt(str(project_dir) + '/aoi/aoi_tmp')

# initialize the class to s1_grd instance
s1_grd = Sentinel1Batch(
    project_dir=project_dir,
    aoi=aoi,
    start=start,
    end=end,
    product_type="GRD",
    beam_mode="IW",
    polarisation="*",
    ard_type="OST-RTC", #Radiometrically Terrain Corrected, images are corrected for radiometric distortions along mountainous slopes
    max_workers=8,
)

# search for files
# added scihub username and pw in Project.py line 260
if not s1_grd.inventory_file:
    s1_grd.search()

# refine inventory
s1_grd.refine_inventory()

# download ascending orbit images
s1_grd.download(s1_grd.refined_inventory_dict[key], concurrent=10, mirror = '2', uname=username, pword=pw)

# set ard parameters
s1_grd.ard_parameters["single_ARD"]["resolution"] = 10  # set output resolution to 100m
s1_grd.ard_parameters["single_ARD"]["remove_speckle"] = True  # apply a speckle filter
s1_grd.ard_parameters["single_ARD"]["to_db"] = True
# s1_grd.ard_parameters["mosaic"]["cut_to_aoi"] = False
# s1_grd.ard_parameters["single_ARD"]["apply_ls_mask"] = True
# s1_grd.ard_parameters["single_ARD"]["to_tif"] = True
# s1_grd.ard_parameters["single_ARD"]["out_projection"] = 3857

# preprocess
s1_grd.grds_to_ards(
    inventory_df=s1_grd.refined_inventory_dict[key],
    timeseries=False,
    timescan=False,
    mosaic=True,
    overwrite=True,
    max_workers=8,
)

# convert to tiff and cut to aoi shape
for u in ['Gamma0_VV_db','Gamma0_VH_db']:
# for u in ['Gamma0_VV','Gamma0_VH']:
    paths = list(Path(project_dir).rglob(str(u + '.img')))
    for i in paths:
        inpath = str(i)
        outpath = str(project_dir) + '/out/' + u + '_' + os.path.split(os.path.split(os.path.split(i)[0])[0])[1] + '.tiff'
        ds = gdal.Open(inpath)
        ds = gdal.Translate(outpath, ds)
        ds = None
        raster.mask_by_shape(outpath,outpath, str(project_dir) + '/aoi/' + aoi_file, rescale=False, description=False, to_db=False)